In [1]:
! pip install dash   
! pip install dash-html-components                                         
! pip install dash-core-components                                     
! pip install plotly

In [3]:
import dash
import dash_html_components as html
import plotly.graph_objects as go
import dash_core_components as dcc
from dash.dependencies import Input, Output
from utils import get_modification_count_per_protein
import pandas as pd
import plotly.express as px

path = r"UHT milk P036.csv"
 
df = pd.read_csv(path)

def count_no_of_modifications(ptm_str):
    #check if NaN value
    if pd.isnull(ptm_str):
        return 0
    return 1 + ptm_str.count(';')

#apply count_no_of_modifications to each PTM column
df['#modifications'] = df['PTM'].apply(count_no_of_modifications)
#print non zero values in modifcations column
df[df['#modifications'] > 0]

app = dash.Dash()

colorscales = px.colors.named_colorscales()

def df_to_plotly(df):
    return {'z': df.values.tolist(),
            'x': df.columns.tolist(),
            'y': df.index.tolist()}

app.layout = html.Div(id = 'parent', children = [
    html.H1(id = 'H1', children = 'Distribution of modifications over different proteins', style = {'textAlign':'center',\
                                            'marginTop':40,'marginBottom':40}),
        dcc.Dropdown( id = 'norm_dropdown',
        style= {'padding': 5,},
        options = [
            {'label':'Normalization: nothing', "value":"" },
            {'label': 'Normalization: total protein modification count', 'value':"protein_total_mod_count"},
            {'label': 'Normalizaton: protein length', 'value':"protein_length"},
            {'label': 'Normalization: protein mass', 'value':"protein_mass"},
            {'label': 'Normalization: protein intensity', 'value':"protein_intensity"},
            ],
        value = ''),
        dcc.Dropdown( id = 'count_dropdown',
        style= {'padding': 5,},
        options = [
            {'label':'Only include proteins with modification count OVER 10', "value":10 },
            {'label': 'Only include proteins with modification count OVER 20', 'value':20},
            {'label': 'Only include proteins with modification count OVER 40', 'value':40},
            {'label': 'Only include proteins with modification count OVER 80', 'value':80},
            {'label': 'Only include proteins with modification count OVER 160', 'value':160},
            {'label': 'Only include proteins with modification count OVER 320', 'value':320},
            {'label': 'Only include proteins with modification count OVER 640', 'value':640},
            ],
        value = 80),
        dcc.Dropdown(
        id='colorscale', 
        style= {'padding': 5,},
        options=[{"value": x, "label": x} 
                 for x in colorscales],
        value='viridis'
        ),
        dcc.Graph(id = 'mod_heatmap_plot'),
                dcc.Loading(
            id="loading-1",
            type="default",
            children=html.Div(id="loading-output-1")
        ),
    ])
    
    
@app.callback(Output(component_id='mod_heatmap_plot', component_property= 'figure'),
              [Input(component_id='norm_dropdown', component_property= 'value'), Input(component_id='count_dropdown', component_property= 'value'), Input("colorscale", "value")])
def graph_update(dropdown_value_norm, dropdown_value_count, scale):
    print(dropdown_value_norm)
    print(dropdown_value_count)
    modPd = pd.DataFrame(get_modification_count_per_protein(df, dropdown_value_count, '{0}'.format(dropdown_value_norm)))
    fig = go.Figure(data=go.Heatmap(df_to_plotly(modPd), colorscale = scale))
    fig.update_layout(xaxis_title = 'Name of protein',
                      yaxis_title = 'Name of modification'
                      )
    return fig  


app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
